# Setup Kubernetes environment for DYNAMOS in FABRIC 

This Jupyter notebook will create the Kubernetes environment in FABRIC after the slice and corresponding nodes have been created.

FABRIC API docs: https://fabric-fablib.readthedocs.io/en/latest/index.html


## Step 1:  Configure the Environment & Create Slice

Before running this notebook, you will need to configure your environment using the [Configure Environment](../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

After following all steps of the Configuring Environment notebook, you should be able to run this notebook without additional steps.

Next, you will need to have setup the slice in FABRIC using the [Create Slice](../create_slice.ipynb) notebook.

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

## Step 2: Setup the Environment for this Notebook

### Step 2.1: Import FABRIC API and other libraries

In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


### Step 2.2: Configure the parameters, get Attributes, and add the nodes
Can be used to set the corresponding slice and other variables used for subsequent cells.

This also prints the necessary information and variables that can be used for later steps.

In [ ]:
# ========== Step 1: Configure Parameters ==========
slice_name = 'DYNAMOS'
# Nodes:
node1_name = 'Node1'
node2_name = 'Node2'
# TODO: more variables and below needed?
# node1_nic_name = 'NIC1'
# node2_nic_name = 'NIC2'
# network_name = 'NET1'

# ========== Step 2: Get and Print Slice and Node Attributes ==========
try:
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)
    # Get slice nodes
    for node in slice.get_nodes():
        print(f"Node: {node.get_name()}")
        # Print full IP address list
        # print(f"  IP Address List: {node.get_ip_addrs()}")
        # Get IP address list
        ip_addrs = node.get_ip_addrs()
        # Loop over them and extract the IPv4 address of the node that is not local
        for interface in ip_addrs:
            addr_info_list = interface.get("addr_info", [])
            for addr in addr_info_list:
                ip = addr.get("local", "")
                if ip and ip != "127.0.0.1" and ":" not in ip:  # Exclude loopback and IPv6
                    print(f"  IPv4 Address: {ip}")
        # print(f"  IP Routes List: {node.get_ip_routes()}")

        # Get the original SSH command
        original_ssh_command = node.get_ssh_command()
        # Print SSH commands to get into the nodes
        print(f"  SSH Command from FABRIC: {original_ssh_command}")
        # Replace the file paths in the SSH command
        updated_ssh_command = original_ssh_command.replace(
            "/home/fabric/work/fabric_config/slice_key", "~/.ssh/slice_key"
        ).replace(
            "/home/fabric/work/fabric_config/ssh_config", "ssh_config"
        )
        # Print the updated SSH command
        print(f"  SSH Command locally (ensuring it is saved according to below steps): {updated_ssh_command}")
        # TODO: probably need to test this locally.
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

#### Run the SSH Commands
To run the SSH Commands follow these steps:
1. From the Jupyter Notebook Hub from FABRIC, download the /fabric_config/fabric_bastion_key, /fabric_config/slice_key and /fabric_config/ssh_config files
2. Add the ssh_config file to this project under /fabric/fabric_config, and change the /fabric_config/ssh_config "IdentityFile" entry to "~/.ssh/fabric_bastion_key", this is the new path to the bastion key of FABRIC from where you will be running the file.

3. Execute these steps to save the SSH files savely on your local machine and avoid problems
```sh
# Open a Linux terminal, such as WSL after opening a CMD in Windows:
wsl
# Navigate to the Downloads directory
cd Downloads
# Create a directory called ssh to store the files
mkdir -p ~/.ssh

# Copy the key files to the SSH directory
cp slice_key ~/.ssh/slice_key
cp fabric_bastion_key ~/.ssh/fabric_bastion_key
# Update permissions
chmod 600 ~/.ssh/slice_key
chmod 600 ~/.ssh/fabric_bastion_key
# Navigate to the SSH directory to verify the files
cd ~/.ssh
# List files including permissions (-l)
ls -l

# Navigate to the fabric_config folder of this project, such as:
cd /mnt/c/Users/cpoet/VSC_Projs/EnergyEfficiency_DYNAMOS/fabric/fabric_config
# Then run the command from the previous step, such as:
ssh -i ~/.ssh/slice_key -F ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe65:a464
# To exit SSH access, type "exit" and press Enter
```
4. Now you can SSH into the nodes using the printed commands.

## Step 3: Configure Kubernetes cluster

This step configures the Kubernetes cluster using scripts in this folder.
TODO: maybe this is done somewhere else, see the README.md, such as from the local node, so try that first.
TODO: if can be done on local machine, use this step to refer to that README.md and execute the commands.

### Configure Control Plane node
TODO: also configure worker node.

In [ ]:
try:
    # TODO: add network here similar to Kubernetes cluster example setup notebook from FABRIC, or is that already done with Kubespray?
    # TODO: change this to below Kubernetes setup.
    # TODO: when changing the slice, such as adding network, it has to be submitted.
    # Submit the updates. Wait until the state is finished and use an interval and a longer timeout to make sure all nodes are created
    slice.submit(wait=True, wait_timeout=1800, wait_interval=10, progress=True, wait_jupyter='text')
    
    # TODO: now also maybe some other steps need to happen before this, and the scripts need to be created to configure the control plane and worker, etc.
    # Upload script file to the nodes
    file_attributes = node1.upload_file(local_file_path="config_control_plane.sh", remote_file_path="config_control_plane.sh")
    # Execute the script
    stdout, stderr = node1.execute(f"chmod +x config_control_plane.sh && ./config_control_plane.sh")

except Exception as e:
    print(f"Exception: {e}")